# Trabajo de Inteligencia artificial
 ## Análisis de noticias

 Realizado por:
 - Marta Aguilar Morcillo
 - Candela Jazmín Gutiérrez González

Fecha: 30/05/2025

Convocatoria de junio.

 ## 1. Lectura de datos

 Se comenzará con la lectura del corpus. Para ello, será necesaria la importación de las siguientes librerías:
 - **nltk:** 
 - **punkt_tab:** para la tokenización de las palabras de los documentos.
 - **contractions:**
 - **sklearn:**

In [2]:
!pip install nltk
import nltk

from nltk import download

download('punkt_tab')                           # Tokenización
nltk.download('averaged_perceptron_tagger')     # POS tagging
nltk.download('averaged_perceptron_tagger_eng') # POS tagging
nltk.download('wordnet')                        # WordNet lemmatizer
nltk.download('omw-1.4')                        # WordNet multilingüe

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [3]:
from nltk.tokenize import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
from nltk.corpus import stopwords
from nltk.data import path
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag

path.append(".")

In [4]:
!pip install contractions
import contractions

In [19]:
import csv
import pandas as pd
from bs4 import BeautifulSoup
from pprint import pprint
import re
from bs4 import MarkupResemblesLocatorWarning
import warnings

In [21]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import recall_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neighbors import KNeighborsClassifier
import spacy

In [22]:
warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)

In [23]:
palabras_vacias_ingles = stopwords.words('english')

In [24]:
nlp = spacy.load("en_core_web_sm")

In [45]:
def elimina_html(contenido):
    return BeautifulSoup(contenido).get_text()

def elimina_no_alfanumerico(contenido):
    return [re.sub(r'[^\w]', '', palabra)
            for palabra in contenido
            if re.search(r'\w', palabra)]

def expandir_constracciones(contenido):
    return contractions.fix(contenido)

def pasar_a_minuscula(contenido):
    return contenido.lower()

def limpiar_texto(texto):
    texto = re.sub(r'[^a-zA-Z\s]', ' ', texto)  # Reemplaza todo lo que no es letra o espacio con espacio
    texto = re.sub(r'\s+', ' ', texto).strip()
    return texto

def elimina_palabras_vacias(contenido):
    return [palabra for palabra in contenido if palabra not in palabras_vacias_ingles]

def lematizador(contenido):
    lemmatizer = WordNetLemmatizer()
    pos_tags = pos_tag(contenido)

    resultado = []
    for palabra, tag in pos_tags:
        if tag.startswith('VB'):  # Verbos
            resultado.append(lemmatizer.lemmatize(palabra, pos='v'))  # infinitivo
        else:  # Sustantivos y el resto tal como están
            resultado.append(palabra)

    return resultado

def extraer_noun_chunks(tokens):
    resultados = []
    doc = nlp(" ".join(tokens))
    
    noun_chunks = [chunk.text.lower().strip() for chunk in doc.noun_chunks if len(chunk.text.split()) <= 3]
    noun_chunks_set = set(noun_chunks)

    i = 0
    while i < len(tokens):
        composed2 = " ".join(tokens[i:i+2]).lower()
        composed3 = " ".join(tokens[i:i+3]).lower()

        if composed3 in noun_chunks_set:
            i += 3  
        elif composed2 in noun_chunks_set:
            i += 2 
        else:
            resultados.append(tokens[i].lower())  
            i += 1

    return resultados + noun_chunks


In [49]:
def proceso_contenido(texto):
    texto = elimina_html(texto)
    texto = expandir_constracciones(texto)
    texto = pasar_a_minuscula(texto)
    texto = limpiar_texto(texto)                # Limpiar antes de tokenizar
    tokens = word_tokenize(texto)               
    tokens = elimina_no_alfanumerico(tokens)    # Limpiar tokens individuales
    tokens = elimina_palabras_vacias(tokens)
    tokens = lematizador(tokens)
    return tokens

In [50]:
# Cargar CSV con codificación segura y punto y coma como separador
df = pd.read_csv("news_corpus.csv", encoding="latin-1", sep=";", quotechar='"')

# Verificar que tenga exactamente 3 columnas
assert df.shape[1] == 3, "El CSV no tiene exactamente 3 columnas. Revisa el separador o las comillas."

resultados = []

for index, fila in df.iterrows():
    autor = [fila.iloc[0]]
    titulo = fila.iloc[1]
    cuerpo = fila.iloc[2]   

    titulo_proc = proceso_contenido(titulo)
    cuerpo_proc = proceso_contenido(cuerpo)

    # Unir las tres listas en una sola lista combinada
    fila_combinada =  autor + titulo_proc + cuerpo_proc

    contenido_final = extraer_noun_chunks(fila_combinada)
    resultados.append(contenido_final)

In [51]:
# Mostrar los primeros 5 documentos procesados
for i, documento in enumerate(resultados[:5]):
    print(f"Documento {i+1}:")
    print(" - Palabras:", documento)
    print()

Documento 1:
 - Palabras: ['chhavi tyagi', 'daman', 'diu', 'revoke', 'mandatory', 'rakshabandhan', 'offices', 'order', 'the', 'daman', 'diu', 'administration', 'wednesday', 'withdraw', 'circular', 'ask', 'women', 'staff', 'tie', 'rakhis', 'male', 'colleagues', 'order', 'trigger', 'backlash', 'employees', 'rip', 'apart', 'the', 'union', 'territory', 'administration', 'force', 'retreat', 'within', 'circular', 'make', 'celebrate', 'it', 'decide', 'celebrate', 'festival', 'in', 'shall', 'remain', 'collectively', 'suitable', 'time', 'wherein', 'shall', 'tie', 'rakhis', 'colleagues', 'order', 'issue', 'august', 'gurpreet', 'singh', 'deputy', 'secretary', 'personnel', 'say', 'to', 'ensure', 'one', 'skipped', 'office', 'attendance', 'report', 'send', 'government', 'next', 'even', 'the', 'two', 'notifications', 'rakshabandhan', 'leave', 'withdraw', 'mandate', 'daman', 'diu', 'administration', 'day', 'apart', 'the', 'circular', 'withdrawn', 'one', 'line', 'order', 'issue', 'late', 'evening', 'ut

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
import numpy as np

corpus = resultados

# 🔖 Temas base con palabras clave iniciales
temas = [ 
    ["child", "family", "welfare"],
    ["technology", "science", "space"],
    ["sports", "match", "score"],
    ["opinion", "public", "speech"],
    ["art", "culture", "awards"],
    ["infrastructure", "urbanism", "transport", "traffic"],
    ["politics", "government", "law"],
    ["justice", "crime"],
    ["conflict", "international", "security"],
    ["religion"],
    ["ecology"],
    ["justice", "crime"],
    ["health", "sickness"]
]

# 📚 Asignación de documentos por tema
asignacion_temas = {
    0: [0, 1, 4, 13, 15],                     # Género, feminismo y derechos de la mujer
    1: [5, 6, 14, 16],                        # Violencia, crimen y justicia
    2: [3, 16, 38, 40],                       # Terrorismo y seguridad nacional
    3: [9, 11, 13],                           # Desinformación, histeria colectiva y rumores
    4: [4, 13, 20, 37],                       # Educación y juventud
    5: [12, 36, 39, 42],                      # Ciencia, salud y tecnología
    6: [7, 8, 19],                            # Protestas sociales y demandas ciudadanas
    7: [6, 18, 22, 41],                       # Infraestructura urbana y transporte
    8: [0, 15, 23, 35],                       # Religión y política identitaria
    9: [10, 26, 28, 39, 44],                  # Medio ambiente y conservación
    10: [0, 5, 6, 13],                        # Corrupción y abusos de poder
    11: [11, 35, 40],                         # Racismo, xenofobia y migración
    12: [7, 8, 19, 22],                       # Economía y precios
    13: [9, 29, 30, 41],                      # Accidentes y desastres
    14: [23, 19, 35],                         # Política y elecciones
    15: [12, 13, 32, 33],                     # Tecnología y privacidad
    16: [1, 18, 42],                          # Cultura y celebridades
    17: [16, 30, 34, 39],                     # Conflictos internacionales
    18: [8, 25, 26, 41],                      # Trabajo y condiciones laborales
    19: [5, 6, 19, 23]                        # Gobierno y ciudadanía
}


# 🔁 Convertimos el corpus a una lista de strings para usar TF-IDF
texts = [" ".join(doc) for doc in corpus]

# TF-IDF para todo el corpus
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(texts)
terms = vectorizer.get_feature_names_out()

# Diccionario para temas expandidos
temas_expandidos = []

# 🔍 Para cada tema, extraer términos dominantes de sus documentos
for idx, doc_ids in asignacion_temas.items():
    if idx >= len(temas):  # Por si hay más asignaciones que temas
        continue
    tema_actual = set(temas[idx])
    
    # Extraer submatriz TF-IDF de los documentos asignados a este tema
    submatrix = X[doc_ids]
    # Calcular media TF-IDF por término en esos documentos
    tfidf_mean = np.asarray(submatrix.mean(axis=0)).flatten()
    # Ordenar términos por TF-IDF medio descendente
    top_indices = tfidf_mean.argsort()[::-1][:20]  # Top 10
    top_terms = set(terms[i] for i in top_indices if terms[i] not in tema_actual)
    
    # Ampliar el tema
    tema_ampliado = tema_actual.union(top_terms)
    temas_expandidos.append(sorted(tema_ampliado))

    print(f"\n🟩 Tema {idx+1}:")
    print(f"   Palabras clave iniciales: {sorted(tema_actual)}")
    print(f"   ➕ Palabras añadidas por TF-IDF: {sorted(top_terms)}")
    print(f"   ✅ Tema final: {sorted(tema_ampliado)}")




🟩 Tema 1:
   Palabras clave iniciales: ['child', 'family', 'welfare']
   ➕ Palabras añadidas por TF-IDF: ['alimony', 'celebrate', 'circular', 'daman', 'diu', 'edit', 'embryos', 'festival', 'gene', 'hotel', 'hotels', 'malaika', 'nota', 'option', 'party', 'rakshabandhan', 'scientists', 'sex', 'signs', 'staff']
   ✅ Tema final: ['alimony', 'celebrate', 'child', 'circular', 'daman', 'diu', 'edit', 'embryos', 'family', 'festival', 'gene', 'hotel', 'hotels', 'malaika', 'nota', 'option', 'party', 'rakshabandhan', 'scientists', 'sex', 'signs', 'staff', 'welfare']

🟩 Tema 2:
   Palabras clave iniciales: ['science', 'space', 'technology']
   ➕ Palabras añadidas por TF-IDF: ['abuse', 'accident', 'athlete', 'buildings', 'court', 'cpwd', 'delhi', 'hussain', 'kumar', 'municipal', 'negligence', 'police', 'quote', 'say', 'snowshoe', 'station', 'suspect', 'unsafe', 'vehicle', 'woman']
   ✅ Tema final: ['abuse', 'accident', 'athlete', 'buildings', 'court', 'cpwd', 'delhi', 'hussain', 'kumar', 'municipa